# Datasets

Contains functions and classes related to datasets and dataloaders

In [ ]:
#| default_exp datasets

In [ ]:
#|export
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
from torch.utils.data import DataLoader, default_collate, Dataset
import fastcore.all as fc

In [ ]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path
import torch
from torch import tensor,nn,optim
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close, equals
from nbdev import show_doc

/Users/johnrichmond/miniconda/envs/miniai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
logging.disable(logging.WARNING)

The dataset management will be developed and demonstrated using databases from Huggingface, but can be readily adapted to work with other datasets.  

## Hugging Face Datasets

Note that a list of the available datasets can be found on the Hugging Face hub: 

[Hugging Face Hub](https://huggingface.co/datasets)

Hugging Face datasets have a dictionary based structure


In [ ]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



The dataset builder has a number of useful properties that can help better understand a dataset.  These are mostly sub properties the info attribute and include:
- features
- splits
- homepage

And lots of other useful things.

Dataset builder allows obtaining details about he dataset without actually downloading it.  Accessing the dataset then uses the load_dataet mehod

In [ ]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [ ]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296607, num_examples=60000, shard_lengths=None, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233810, num_examples=10000, shard_lengths=None, dataset_name='fashion_mnist')}

In [ ]:
ds_builder.info.homepage

'https://github.com/zalandoresearch/fashion-mnist'

The load_dataset function can be used to load the whole dataset, which can then be sub divided if necessary

In [ ]:
ds_loader = load_dataset(name)

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 332.45it/s]


The different splits can then be assigned separately

In [ ]:
train_ds, test_ds = ds_loader['train'], ds_loader['test']

Retireving individual items can be done using the dataset features, in this case the image and label

In [ ]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [ ]:
train_ds[0]['image']

This can be simplified by assigning the features (keys) to x and y

In [ ]:
x, y = ds_builder.info.features
x, y

('image', 'label')

In [ ]:
img = train_ds[0][x]
img

In [ ]:
#|export
def inplace(f):
    """ This function allows a function that does not return anything directly (ie one that modifies things
    without a return statenent) to then be used in an application that required a return.  So this function
    is a wrapper of another function that simply executes the function and then returns the modified input
    """
    def _f(b):
        f(b)
        return b
    return _f

Test that the inline function does as it should

In [ ]:
@inplace
def torch_cos(x):
    # Modify in place x to become cos(x)
    torch.cos_(x)

In [ ]:
angle = torch.tensor(torch.pi/8)
y = torch_cos(angle)
equals(y, torch.cos(torch.tensor(torch.pi/8)))

True

The HuggingFace dataset provides datasets that return a dictionary of features.  It also provides the capability to apply transforms to the dataset items using the with_transforms method.  In the case of the Fashion Mnist dataset it is convenient to be able to return the inputs and targets as flattened tensors.  This can be done using a function to convert items to tensors and then flatten them, followed by a collation function that will build upon the default_collate function from Pytorch

In [ ]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

To illustrate how default_collate will deal with a dict see the following

In [ ]:
batch = dict(a=[1],b=[2]), dict(a=[3],b=[4])
default_collate(batch)

{'a': [tensor([1, 3])], 'b': [tensor([2, 4])]}

In [ ]:
# create a dataset that will apply the transformi function to the items it returns
tsds = train_ds.with_transform(transformi)

In [ ]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [ ]:
tsds[0]

{'image': tensor([0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,
         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.29, 0.00, 0.00, 0.00, 0.02, 0.00, 0.00, 0.00, 0.00, 0.00,
         0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01, 0.00, 0.14, 0.53, 0.50, 0.24, 0.21, 0.00,
         0.00, 0.00, 0.00, 0.01, 0.02, 0.00, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.00,
    

The dictionary aspects of this are not needed and so the collate_dict function that follows extracts the individual items of the batch and returns then as a tuple of the input and output tensors

In [ ]:
#|export
def collate_dict(ds):
    """ when a dataset is defined by a dictionary this will identify the features and split into inputs and outputs
    as tensor arrays ready for input to a model
    """
    get = itemgetter(*ds.features)
    def _f(b):
        """ return a tuple containing the values associated with each of the keys returned by the itemgetter 
        given that default_collate returns a dict with two keys and a stacked tensor for each batch"""
        return get(default_collate(b))
    return _f

Note that it is necessary initialise the collate_dict function supplied to the dataloader with the dataset.  This is necessary so that the embeded get function can identify the features to be loaded.  It would be nice if this could be avoided

In [ ]:
train_dl = DataLoader(tsds, batch_size=4, shuffle=True, collate_fn=collate_dict(tsds))

In [ ]:
xb, yb = next(iter(train_dl))

In [ ]:
xb.shape, yb.shape

(torch.Size([4, 784]), torch.Size([4]))

In [ ]:
def collate_dict2(batch):
    pairs = default_collate(batch)
    items = [values for values in pairs.values()]
    return items

In [ ]:
train_dl2 = DataLoader(tsds, batch_size=4, shuffle=True, collate_fn=collate_dict2)

In [ ]:
xb, yb = next(iter(train_dl2))

In [ ]:
xb.shape, yb.shape

(torch.Size([4, 784]), torch.Size([4]))

### Add Basic DataLoader Helper Functions

In [ ]:
#|export
def get_dls(train_ds, # Training Dataset
            valid_ds, # Validation Dataset
            bs=64, # Batch size (int)
            **kwargs):
    """ Utility function to return train and validation dataloaders from two datasets
    """
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs)
    valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False, **kwargs)
    return train_dl, valid_dl

In [ ]:
#|export
class DataLoaders:
    """ Convenience class to create and contain a collection of datasets
    """
    
    def __init__(self, *dls): self.train,self.valid = dls[:2]

    @classmethod
    def from_dd(cls, 
                dd, # dataset with splits identified by dictionary keys, such as Huggingface
                batch_size: int=64, # Batch size (int)
                **kwargs):
        """Create dataloaders based upon dictionary based datasets (such as HuggingFace).  This is 
        simply a shortcut to avoid creating the dataloaders individually.  Returns a dataloader
        for each the train and validation splits.  In future could be adapted to cope with more splits
        """
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()